In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import math
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import re


In [39]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device in use: {device}')

Device in use: cpu


In [51]:
import re
from collections import Counter

class SimpleTokenizer:
    def __init__(self):
        self.vocab = {}
        self.rev_vocab = {}
        self.token_count = 0

    def bulid_vocab(self, texts,min_freq=2):
        all_chars=[]
        for text in texts:
            cleaned_text =re.sub(r'[^a-zA-Z0-9\s]', '', text)     
            all_chars.extend(list(cleaned_text))

        char_counts = Counter(all_chars)

        # Add sppecial tokens
        self.vocab= {
            '<pad>': 0,
            '<unk>': 1,
            '<sos>': 2,
            '<eos>': 3
        }
        self.inx_to_token = {0: '<pad>', 1: '<unk>', 2: '<sos>', 3: '<eos>'}

        indx=4
        for char,count in char_counts.items():
            if count >= min_freq:
                self.vocab[char] = indx
                self.inx_to_token[indx] = char
                indx += 1    
            
            def encode(self,token):    
                tokens =[self.vocab['<bos>']]
                for char in text:
                    tokens.append(self.vocab.get(char, self.vocab['<unk>']))
                tokens.append(self.vocab['<eos>'])
                return tokens
            
            def decode(self,token_ids):
                text = ""
                for token_id in token_ids:
                    if token_id in [self.vocab['<pad>'],self.vocab['<bps>'],self.vocab['<eos>']]:
                        continue
                    elif token_id == self.vocab['<unk>']:
                        text += '?'
                    else:
                        text += self.inx_to_token(token_id, '')
                return text




In [52]:
sample_texts_hi = [
    "नमस्ते, आप कैसे हैं?",
    "यह एक साधारण उदाहरण है।",
    "पायथन प्रोग्रामिंग भाषा बहुत लोकप्रिय है।",
    "मुझे किताबें पढ़ना पसंद है।",
    "आपका दिन शुभ हो!",
    "क्या आप चाय पीना चाहेंगे?",
    "हिंदी भाषा में संवाद करना अच्छा लगता है।",
    "यहां कुछ और वाक्य हैं।",
    "आइए हम एक साथ सीखते हैं।",
    "आपका स्वागत है!",
    "मौसम आज बहुत सुहाना है।",
    "मैं हर दिन कुछ नया सीखता हूँ।",
    "क्या आप मेरी मदद कर सकते हैं?",
    "कृपया दरवाज़ा बंद कर दें।",
    "मुझे संगीत सुनना अच्छा लगता है।",
    "हम कल बाजार जाएंगे।",
    "तुम क्या कर रहे हो?",
    "यह खाना बहुत स्वादिष्ट है।",
    "कृपया मुझे माफ़ कर दें।",
    "आज स्कूल में छुट्टी है।",
    "तुम्हारा नाम क्या है?",
    "मुझे हिंदी में लिखना पसंद है।",
    "मैं दिल्ली में रहता हूँ।",
    "हम जल्दी मिलेंगे।",
    "यह बहुत कठिन सवाल है।",
    "यह समाधान सही लगता है।",
    "कृपया धीरे बोलिए।",
    "मैं थोड़ा व्यस्त हूँ।",
    "आपका उत्तर सही है।",
    "क्या आप मुझे सुन सकते हैं?",
    "मैंने यह किताब पढ़ी है।",
    "यह दृश्य बहुत सुंदर है।",
    "आज बहुत गर्मी है।",
    "मैं शाम को टहलने जाता हूँ।",
    "क्या आप खेलना पसंद करते हैं?",
    "मुझे आपकी बात पसंद आई।",
    "आपका मोबाइल कहां है?",
    "कृपया लाइट बंद कर दें।",
    "मैं कल फिर आऊंगा।",
    "हम साथ में फिल्म देखेंगे।",
    "आपका पसंदीदा रंग कौन सा है?",
    "मुझे नीला रंग पसंद है।",
    "कृपया अपना नाम बताइए।",
    "क्या आप तैयार हैं?",
    "मैंने अपना होमवर्क पूरा कर लिया।",
    "यह बहुत अच्छा विचार है।",
    "आप बहुत अच्छे दोस्त हैं।",
    "क्या आप खाना खा चुके हैं?",
    "मैंने नाश्ता कर लिया है।",
    "क्या हम अब शुरू करें?",
    "मुझे आपके साथ काम करना अच्छा लगा।",
    "हमने बहुत मज़ा किया।",
    "क्या आप मुझे देख सकते हैं?",
    "मैं अभ्यास कर रहा हूँ।",
    "आज सोमवार है।",
    "कल रविवार था।",
    "मुझे गर्म दूध पसंद है।",
    "कृपया दरवाज़ा खोलिए।",
    "यह गीत बहुत मधुर है।",
    "मैंने एक सुंदर सपना देखा।",
    "हम पार्क में खेलते हैं।",
    "तुम बहुत होशियार हो।",
    "यह तो बहुत आसान है।",
    "क्या तुम मुझे पहचानते हो?",
    "मैं रोज़ व्यायाम करता हूँ।",
    "मुझे फल खाना पसंद है।",
    "मैंने अपना काम खत्म कर लिया।",
    "कृपया थोड़ी देर रुकिए।",
    "क्या यह आपकी किताब है?",
    "मुझे ठंड लग रही है।",
    "मैं कुछ कहना चाहता हूँ।",
    "क्या आप अंग्रेज़ी बोलते हैं?",
    "मैं थोड़ा बहुत जानता हूँ।",
    "आपका दिन कैसा रहा?",
    "मुझे थोड़ी मदद चाहिए।",
    "यह मेरा पसंदीदा खेल है।",
    "मैं आज खुश हूँ।",
    "कृपया ध्यान दीजिए।",
    "मुझे हिंदी में बोलना अच्छा लगता है।",
    "यह समय बहुत महत्वपूर्ण है।",
    "मैं नए शब्द सीख रहा हूँ।",
    "आपके विचार प्रेरणादायक हैं।",
    "मैंने अपनी योजना बदल दी है।",
    "तुम बहुत अच्छे दिख रहे हो।",
    "क्या हम मिल सकते हैं?",
    "मुझे थोड़ा विश्राम चाहिए।",
    "यह स्थान बहुत शांत है।",
    "हम जल्दी निकलेंगे।",
    "तुम कब वापस आओगे?",
    "मैं कल यात्रा पर जा रहा हूँ।",
    "आपका स्वास्थ्य कैसा है?",
    "मुझे नींद आ रही है।",
    "आपका पसंदीदा खाना क्या है?",
    "मैंने अभी खाना खाया है।",
    "आपकी मुस्कान बहुत प्यारी है।",
    "मैंने यह पहले सुना है।",
    "क्या आप दोहराएंगे?",
    "मैं समझ नहीं पाया।",
    "कृपया फिर से कहिए।",
    "यह जानकारी उपयोगी है।",
    "मुझे यात्रा करना पसंद है।",
    "मैं अभी रास्ते में हूँ।",
    "क्या आपको मेरी आवाज़ आ रही है?",
    "मैं आपसे सहमत हूँ।"
]


In [53]:
tokenizer= SimpleTokenizer()
tokenizer.bulid_vocab(sample_texts_hi)
print("Vocabulary size:", len(tokenizer.vocab))

Vocabulary size: 5


In [54]:
class TestDataset(Dataset):
    def __init__(self,texts,tokenizer,max_length=50):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = []
        for text in texts:
            tokens = tokenizer.encode(text)
            if len(tokens) > max_length:
                tokens = tokens[:max_length]
            else:
                tokens += [tokenizer.vocab['<pad>']] * (max_length - len(tokens))
            self.data.append(tokens)

    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        tokens = self.data[idx]
        input_ind= torch.tensor(tokens[:-1], dtype=torch.long)
        target_ind = torch.tensor(tokens[1:], dtype=torch.long)
        return input_ind, target_ind

In [55]:
dataset=  TestDataset(sample_texts_hi, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)                                   

AttributeError: 'SimpleTokenizer' object has no attribute 'encode'